In [86]:
import pandas as pd
import numpy as np

In [441]:
sample = pd.read_csv("~/Downloads/sample_submission.csv")

In [383]:
test_set = pd.read_csv("~/Downloads/test_kaggle.csv")
test_set

,id,user_id,item_id,context_feature_id
0,0,4,16835,2
1,1,4,22590,3
2,2,4,1978,1
3,3,4,28916,1
4,4,4,14427,2
...,...,...,...,...
381380,381380,200151,1702,1
381381,381381,200151,21632,1
381382,381382,200151,30477,1
381383,381383,200151,30477,1


In [9]:
df_train = pd.read_csv("~/Downloads/training.csv")
df_train

,user_id,item_id,context_feature_id
0,0,28366,2
1,0,16109,2
2,0,11500,3
3,0,20750,2
4,0,8759,2
...,...,...,...
970240,200152,30710,2
970241,200152,30710,2
970242,200152,12006,2
970243,200152,25030,2


In [10]:
df_feature = pd.read_csv("~/Downloads/item_feature.csv")
df_feature

,item_id,item_feature_id
0,0,139
1,1,55
2,2,11
3,3,138
4,4,138
...,...,...
39896,39896,138
39897,39897,95
39898,39898,142
39899,39899,131


### Merge two dataframe based on "item_id" - the new dataframe called "df_full"

In [68]:
df_full = df_train.merge(df_feature , how='left', on='item_id')
df_full

,user_id,item_id,context_feature_id,item_feature_id
0,0,28366,2,7
1,0,16109,2,7
2,0,11500,3,7
3,0,20750,2,7
4,0,8759,2,7
...,...,...,...,...
970240,200152,30710,2,148
970241,200152,30710,2,148
970242,200152,12006,2,148
970243,200152,25030,2,148


### Only keep user and item two columns - the new dataframe called "df_user_item"

In [70]:
df_user_item = df_full[['user_id','item_id']]
df_user_item

,user_id,item_id
0,0,28366
1,0,16109
2,0,11500
3,0,20750
4,0,8759
...,...,...
970240,200152,30710
970241,200152,30710
970242,200152,12006
970243,200152,25030


In [77]:
# drop duplicate rows
df_user_item = df_user_item.drop_duplicates()

### Below is some EDA proess

In [78]:
len(df_full['user_id'].unique())

169698

In [79]:
len(df_full['item_id'].unique())

37978

In [80]:
len(df_full['context_feature_id'].unique())

4

In [81]:
len(df_full['item_feature_id'].unique())

183

In [115]:
df_full['item_id'].unique().max()

39900

## Negative Sampling 

### User-based (Randomly add same number of zeros based on number of ones per user)

In [88]:
df_user_item['binary_ind'] = np.array([1]*len(df_user_item))

In [94]:
df_user_item

,user_id,item_id,binary_ind
0,0,28366,1
1,0,16109,1
2,0,11500,1
3,0,20750,1
4,0,8759,1
...,...,...,...
970239,200150,20387,1
970240,200152,30710,1
970242,200152,12006,1
970243,200152,25030,1


In [145]:
import random

In [122]:
#negative sampling - add zeros
user_list = []
rand_item_list = []
for user_id in df_user_item['user_id'].unique():
    list_item_id = df_user_item[df_user_item['user_id'] == user_id]['item_id'].values
    rand_item_list.append(random.sample(range(0, 39901), len(list_item_id)))
    user_list.append(user_id)

In [126]:
#create dataframe with zeros
df_user_neg_sam = pd.DataFrame({'user_id':user_list,'item_id':rand_item_list})
df_user_neg_sam = df_user_neg_sam.explode('item_id')
df_user_neg_sam['binary_ind'] = np.array([0]*len(df_user_neg_sam))

In [195]:
#combine two dataframes
df_new = df_user_item.append(df_user_neg_sam)
df_new.groupby(['user_id','item_id']).size().value_counts

<bound method IndexOpsMixin.value_counts of user_id  item_id
0        2694       1
         4707       1
         8759       1
         11500      1
         12085      1
                   ..
200152   25030      1
         30710      1
         31264      1
         31962      1
         35679      1
Length: 1725844, dtype: int64>

In [196]:
#check if there're duplicate rows of user_id and item_id
df_new.duplicated(subset=['user_id', 'item_id']).sum()

142

In [197]:
#drop duplicate rows
df_new = df_new.drop_duplicates(subset=['user_id', 'item_id'])

In [198]:
#check if there're duplicate rows of user_id and item_id and make sure the output is zero
df_new.duplicated(subset=['user_id', 'item_id']).sum()

0

In [284]:
#count '1's in the dataframe
df_new[df_new['binary_ind']==1].count()

user_id       862993
item_id       862993
binary_ind    862993
dtype: int64

In [285]:
#count '0's in the dataframe
df_new[df_new['binary_ind']==0].count()

user_id       862851
item_id       862851
binary_ind    862851
dtype: int64

### Create Sparse Matrix and U,V vector

In [286]:
import scipy.sparse as sparse
user_items = sparse.csr_matrix((df_new['binary_ind'],(df_new['user_id'], df_new['item_id'])))

In [287]:
user_vector = df_new['user_id'].values
item_vector = df_new['item_id'].values

# Encoding data

In [298]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df_new, test_size=0.2, random_state=42)

In [299]:
len(train['user_id'].unique())

169548

In [300]:
train_user_ids = np.sort(np.unique(train.user_id.values))
train_user_ids[:15]

array([ 0,  1,  2,  3,  5,  7,  8, 10, 12, 14, 15, 17, 18, 19, 20])

In [301]:
num_users = len(train_user_ids)
num_users

169548

In [302]:
userid2idx = {o:i for i,o in enumerate(train_user_ids)}

In [303]:
train["user_id"] = train["user_id"].apply(lambda x: userid2idx[x])
train.head()

<ipython-input-303-7c6e48d18fa9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["user_id"] = train["user_id"].apply(lambda x: userid2idx[x])


,user_id,item_id,binary_ind
141376,141253,10972,0
90558,90481,30211,0
137060,23984,12757,1
33879,33849,21735,0
254450,44429,400,1


In [304]:
val["user_id"] = val["user_id"].apply(lambda x: userid2idx.get(x, -1)) # -1 for users not in training
val.head()

<ipython-input-304-66fe1848a552>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val["user_id"] = val["user_id"].apply(lambda x: userid2idx.get(x, -1)) # -1 for users not in training


,user_id,item_id,binary_ind
354800,61896,17445,1
894291,156259,12016,1
921981,161052,24590,1
611726,106829,10732,1
94113,94033,29537,0


In [305]:
val = val[val["user_id"] >= 0].copy()
val.head()

,user_id,item_id,binary_ind
354800,61896,17445,1
894291,156259,12016,1
921981,161052,24590,1
611726,106829,10732,1
94113,94033,29537,0


In [307]:
# now encoding movieId
train_item_ids = np.sort(np.unique(train.item_id.values))
num_items = len(train.item_id)
print(num_items)
train_item_ids[:15]

1380675


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [308]:
itemid2idx = {o:i for i,o in enumerate(train_item_ids)}
train["item_id"] = train["item_id"].apply(lambda x: itemid2idx[x])
val["item_id"] = val["item_id"].apply(lambda x: itemid2idx.get(x, -1))

<ipython-input-308-368d636eef53>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["item_id"] = train["item_id"].apply(lambda x: itemid2idx[x])


In [309]:
val = val[val["item_id"] >= 0].copy()
val.head()

,user_id,item_id,binary_ind
354800,61896,17445,1
894291,156259,12016,1
921981,161052,24590,1
611726,106829,10732,1
94113,94033,29537,0


In [310]:
val.shape

(344853, 3)

# Embedding layer

In [311]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [312]:
embed = nn.Embedding(169698, 1)
embed.weight

Parameter containing:
tensor([[-0.1670],
        [ 0.5097],
        [ 1.7602],
        ...,
        [ 1.3954],
        [-2.6201],
        [ 2.2207]], requires_grad=True)

# Matrix factorization model

In [373]:
class MF(nn.Module):
    def __init__(self, num_users, num_items, emb_size=1):
        super(MF, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        # initlializing weights
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        
    def forward(self, u, v):
        u = self.user_emb(u)
        v = self.item_emb(v)
        return torch.sigmoid((u*v).sum(1)) 

In [314]:
# class MF(nn.Module):
#     def __init__(self, num_users, num_items, emb_size=100, seed=23):
#         super(MF, self).__init__()
#         torch.manual_seed(seed)
#         self.user_emb = nn.Embedding(num_users, emb_size)
#         self.user_bias = nn.Embedding(num_users, 1)
#         self.item_emb = nn.Embedding(num_items, emb_size)
#         self.item_bias = nn.Embedding(num_items, 1)
#         # init 
#         self.user_emb.weight.data.uniform_(0,0.05)
#         self.item_emb.weight.data.uniform_(0,0.05)
#         self.user_bias.weight.data.uniform_(-0.01,0.01)
#         self.item_bias.weight.data.uniform_(-0.01,0.01)
#         #print(self.user_emb.weight.data.uniform_(0,0.05))
#     def forward(self, u, v):
#         ### BEGIN SOLUTION
#         U = self.user_emb(u)
#         V = self.item_emb(v)
#         b_u = self.user_bias(u).squeeze()
#         b_v = self.item_bias(v).squeeze()
#         #return (U*V).sum(1) +  b_u  + b_v
#         return torch.sigmoid((U*V).sum(1) +  b_u  + b_v)

# Debugging MF model

In [315]:
users = torch.LongTensor(df_new.user_id.values)
users

tensor([     0,      0,      0,  ..., 200152, 200152, 200152])

In [316]:
items = torch.LongTensor(df.item_id.values)
items

tensor([28366, 16109, 11500,  ..., 12006, 25030, 14578])

In [321]:
num_users = len(train_user_ids)
num_users

169548

In [322]:
num_items = len(train_item_ids)
num_items

39901

In [334]:
# num_users = 200153
# num_items = 39901
emb_size = 1
num_users = 169548
num_items = 39901
user_emb = nn.Embedding(num_users, emb_size)
item_emb = nn.Embedding(num_items, emb_size)
users = torch.LongTensor(df_new.user_id.values)
items = torch.LongTensor(df_new.item_id.values)

# Training MF model

In [217]:
num_users = len(train.user_id.unique())
num_items = len(train.item_id.unique())
print(num_users, num_items) 

169506 39901


In [225]:
def train_one_epoch(model, train_df, optimizer):
    """ Trains the model for one epoch"""
    model.train()
    users = torch.LongTensor(train_df.user_id.values)
    items = torch.LongTensor(train_df.item_id.values)
    ratings = torch.FloatTensor(train_df.binary_ind.values)
    y_hat = model(users, items)
    loss = criterion(torch.squeeze(y_hat), ratings)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss = criterion(torch.squeeze(y_hat), ratings).item()
    return train_loss

In [226]:
def valid_metrics(model, valid_df):
    """Computes validation loss and accuracy"""
    model.eval()
    users = torch.LongTensor(valid_df.user_id.values) 
    items = torch.LongTensor(valid_df.item_id.values) 
    ratings = torch.FloatTensor(valid_df.binary_ind.values)
    y_hat = model(users, items)
    valid_loss = criterion(torch.squeeze(y_hat), ratings).item()
    predicted_test = y_hat.round().detach().numpy()
    correct_test = np.sum(predicted_test == ratings.detach().numpy())
    valid_acc = correct_test/len(valid_df)
    return valid_loss, valid_acc



In [227]:
def training(model, train_df, valid_df, epochs=10, lr=0.01, wd=0.0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    for i in range(epochs):
        train_loss = train_one_epoch(model, train_df, optimizer)
        valid_loss, valid_acc = valid_metrics(model, valid_df) 
        print("train loss %.3f valid loss %.3f valid acc %.3f" % (train_loss, valid_loss, valid_acc)) 

In [325]:
num_users = len(train_user_ids)
num_users

169548

In [327]:
num_items = len(train_item_ids)
num_items

39901

In [328]:
model = MF(num_users, num_items, emb_size=1)

In [333]:
criterion = torch.nn.BCELoss()
training(model, train, val, epochs=10, lr=0.1, wd=1e-5)

train loss 0.656 valid loss 0.629 valid acc 0.854
train loss 0.625 valid loss 0.601 valid acc 0.857
train loss 0.596 valid loss 0.581 valid acc 0.855
train loss 0.567 valid loss 0.564 valid acc 0.852
train loss 0.543 valid loss 0.548 valid acc 0.853
train loss 0.524 valid loss 0.533 valid acc 0.856
train loss 0.510 valid loss 0.518 valid acc 0.856
train loss 0.498 valid loss 0.506 valid acc 0.857
train loss 0.489 valid loss 0.498 valid acc 0.858
train loss 0.482 valid loss 0.493 valid acc 0.858


In [337]:
test_set

,id,user_id,item_id,context_feature_id
0,0,4,16835,2
1,1,4,22590,3
2,2,4,1978,1
3,3,4,28916,1
4,4,4,14427,2
...,...,...,...,...
381380,381380,200151,1702,1
381381,381381,200151,21632,1
381382,381382,200151,30477,1
381383,381383,200151,30477,1


In [347]:
test_set['user_id'].unique() == df_train['user_id'].unique()

<ipython-input-347-0f5e28791e8c>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  test_set['user_id'].unique() == df_train['user_id'].unique()


False

In [352]:
len(set(test_set['user_id'].unique()).intersection(set(df_train['user_id'].unique())))/len(test_set['user_id'].unique())

0.19628955216002955

In [401]:
test_set['binary_ind'] = np.array([1]*len(test_set))

In [402]:
test_set

,id,user_id,item_id,context_feature_id,user_id_,item_id_,binary_ind
0,0,4,16835,2,-1,16835,1
1,1,4,22590,3,-1,22590,1
2,2,4,1978,1,-1,1978,1
3,3,4,28916,1,-1,28916,1
4,4,4,14427,2,-1,14427,1
...,...,...,...,...,...,...,...
381380,381380,200151,1702,1,-1,1702,1
381381,381381,200151,21632,1,-1,21632,1
381382,381382,200151,30477,1,-1,30477,1
381383,381383,200151,30477,1,-1,30477,1


In [403]:
test_s_matrix = sparse.csr_matrix((test_set['binary_ind'],(test_set['user_id'], test_set['item_id'])))

In [419]:
test_set["user_id"] = test_set["user_id"].apply(lambda x: userid2idx.get(x, -1))
test_set = test_set[test_set["user_id"] >= 0].copy()
#test_set_neg = test_set[test_set["user_id"] < 0].copy()
test_set["item_id"] = test_set["item_id"].apply(lambda x: itemid2idx.get(x, -1))
test_set = test_set[test_set["item_id"] >= 0].copy()
#test_set_neg_item = test_set_pos[test_set_pos["user_id"] < 0].copy()
#test_set_neg = test_set_neg.append(test_set_neg_item)

In [421]:
test_set

,id,user_id,item_id,context_feature_id,user_id_,item_id_,binary_ind
30,30,7,25869,1,7,25869,1
31,31,7,12187,1,7,12187,1
32,32,7,32370,1,7,32370,1
33,33,7,11400,1,7,11400,1
34,34,7,25869,1,7,25869,1
...,...,...,...,...,...,...,...
381370,381370,169546,20571,1,169546,20571,1
381371,381371,169546,20387,1,169546,20387,1
381372,381372,169546,7953,1,169546,7953,1
381373,381373,169546,7068,1,169546,7068,1


In [422]:
reversed_userid2idx = {value : key for (key, value) in userid2idx.items()}
reversed_itemid2idx = {value : key for (key, value) in itemid2idx.items()}

In [425]:
def valid_metrics(model, test_df_pos):
    """Computes validation loss and accuracy"""
    model.eval()
    users = torch.LongTensor(test_df_pos.user_id.values) 
    items = torch.LongTensor(test_df_pos.item_id.values) 
    ratings = torch.FloatTensor(test_df_pos.binary_ind.values)
    y_hat_pos = model(users, items)
    #y_hat_neg = [np.mean(y_hat_pos.detach().numpy())]*len(test_df_neg)
    test_df_pos['rating'] = y_hat_pos.detach().numpy()
    test_df_pos['index_'] = [reversed_userid2idx[i] for i in test_df_pos['user_id']]
    #test_df_neg['rating'] = y_hat_neg
    #test_df_neg['index_'] = [reversed_itemid2idx[i] for i in test_df_neg['user_id']]
    #test_set = test_df_neg.append(test_df_pos)
    #test_set = test_set.sort_values(by=['index_'])
    #valid_loss = criterion(torch.squeeze(y_hat), ratings).item()
    #predicted_test = y_hat.round().detach().numpy()
    #correct_test = np.sum(predicted_test == ratings.detach().numpy())
    #valid_acc = correct_test/len(valid_df)  
    return test_df_pos, np.mean(y_hat_pos.detach().numpy())
    #return valid_loss, valid_acc, test_df_pos

In [427]:
df_pos,mean = valid_metrics(model, test_set)

In [428]:
df_pos

,id,user_id,item_id,context_feature_id,user_id_,item_id_,binary_ind,rating,index_
30,30,7,25869,1,7,25869,1,0.704213,10
31,31,7,12187,1,7,12187,1,0.344937,10
32,32,7,32370,1,7,32370,1,0.340187,10
33,33,7,11400,1,7,11400,1,0.348476,10
34,34,7,25869,1,7,25869,1,0.704213,10
...,...,...,...,...,...,...,...,...,...
381370,381370,169546,20571,1,169546,20571,1,0.411582,200150
381371,381371,169546,20387,1,169546,20387,1,0.745691,200150
381372,381372,169546,7953,1,169546,7953,1,0.577797,200150
381373,381373,169546,7068,1,169546,7068,1,0.744257,200150


In [429]:
test_set = pd.read_csv("~/Downloads/test_kaggle.csv")
test_set

,id,user_id,item_id,context_feature_id
0,0,4,16835,2
1,1,4,22590,3
2,2,4,1978,1
3,3,4,28916,1
4,4,4,14427,2
...,...,...,...,...
381380,381380,200151,1702,1
381381,381381,200151,21632,1
381382,381382,200151,30477,1
381383,381383,200151,30477,1


In [438]:
merged = test_set.merge(df_pos, left_index=True, right_index=True,how='outer')
merged["rating"].fillna(mean, inplace = True)
merged

,id_x,user_id_x,item_id_x,context_feature_id_x,id_y,user_id_y,item_id_y,context_feature_id_y,user_id_,item_id_,binary_ind,rating,index_
0,0,4,16835,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.568628,NaN
1,1,4,22590,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.568628,NaN
2,2,4,1978,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.568628,NaN
3,3,4,28916,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.568628,NaN
4,4,4,14427,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.568628,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381380,381380,200151,1702,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.568628,NaN
381381,381381,200151,21632,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.568628,NaN
381382,381382,200151,30477,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.568628,NaN
381383,381383,200151,30477,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.568628,NaN


In [440]:
final_result = merged[['id_x','rating']]
final_result

,id_x,rating
0,0,0.568628
1,1,0.568628
2,2,0.568628
3,3,0.568628
4,4,0.568628
...,...,...
381380,381380,0.568628
381381,381381,0.568628
381382,381382,0.568628
381383,381383,0.568628


In [450]:
hard_pred_result = []
for rating in final_result['rating']:
    if rating >= 0.5:
        hard_pred_result.append(1)
    else:
        hard_pred_result.append(0)
hard_pred_sample = []
for rating in sample['rating']:
    if rating >= 0.5:
        hard_pred_sample.append(1)
    else:
        hard_pred_sample.append(0)

In [452]:
np.sum(np.array(hard_pred_result)==np.array(hard_pred_sample)) / len(sample)

0.9225402152680362